# Chap04 - Word2Vec 속도 개선

[Chap03](https://github.com/ExcelsiorCJH/DLFromScratch2/tree/master/Chap03-Word2Vec)에서 살펴본 CBOW 모델의 구조는 다음과 같은 문제가 있다.

- 말뭉치(corpus)에 포함된 어휘 수가 많아지면 계산량이 커진다.

이를 해결하기 위해 이번 장에서는 두 가지 개선을 추가한다.

1. `Embedding` 레이어를 도입한다.
2. 네거티브 샘플링(NEG, Negative Sampling)이라는 새로운 손실함수를 도입한다.

## 4.1 Word2Vec 개선 ①

[Chap03](https://github.com/ExcelsiorCJH/DLFromScratch2/tree/master/Chap03-Word2Vec)에서 구현한 CBOW 모델의 문제점은 아래의 그림에서 확인할 수 있듯이 예를 들어, 어휘가 100만개, 은닉층의 뉴런이 100개인 CBOW 모델의 경우에는 다음의 두 계산이 병목(bottleneck)이 된다.

- 입력층의 원핫(one-hot) 표현과 가중치 행렬 $\mathbf{W}_{\text{in}}$의 곱 계산

- 은닉층과 가중치 행렬 $\mathbf{W}_{\text{out}}$의 곱 및 `Softmax`(특히 분모)의 계산

<img src="./images/cbow_big.png" width="65%" height="65%" />

### 4.1.1 Embedding 계층

각 단어(어휘)를 원핫 표현으로 변환한 다음 가중치 행렬을 곱해주는 작업은 **결과적으로 단지 각 단어에 해당하는 특정 행을 추출**하는 것 뿐이다. 따라서, 원핫 표현으로의 변환과 가중치 행렬 곱 계산은 사실상 필요하지 않다.

![](./images/embedding.png)

### 4.1.2 Embedding 계층 구현

In [2]:
import numpy as np

W = np.arange(21).reshape(7, 3)
W

array([[ 0,  1,  2],
       [ 3,  4,  5],
       [ 6,  7,  8],
       [ 9, 10, 11],
       [12, 13, 14],
       [15, 16, 17],
       [18, 19, 20]])

In [3]:
# 두 번째 행 가져오기
# -> index=2에 해당하는 단어 벡터
W[2]

array([6, 7, 8])

In [4]:
W[5]

array([15, 16, 17])

In [5]:
# 가중치 W로 부터 여러행을 
# 한꺼번에 추출하는 예제
idx = np.array([1, 0, 3, 0])
W[idx]

array([[ 3,  4,  5],
       [ 0,  1,  2],
       [ 9, 10, 11],
       [ 0,  1,  2]])

In [7]:
# Embedding Layer 구현
# commons/layers.py
class Embedding:
    def __init__(self, W):
        self.params = [W]
        self.grads = [np.zeros_like(W)]
        self.idx = None
        
    def forward(self, idx):
        W, = self.params
        self.idx = idx
        out = W[idx]
        return out
    
    def backward(self, dout):
        dW, = self.grads
        dW[...] = 0
        np.add.at(dW, self.idx, dout)
        return None

## 4.2 Word2Vec 개선 ②

4.1에서는 `Input - hidden`의 병목을 `Embedding`이라는 새로운 레이어를 도입해주면서 해결했고, 이번에는 `hidden - output`의 병목을 **Negative Sampling**을 통해 해결한다.

### 4.2.1 은닉층 이후 계산의 문제점

아래의 그림처럼 100만개의 단어에 대해 Softmax를 구하게 되면 계산량이 많아지는 문제가 있다.


$$
y_k = \frac{\exp{(s_k)}}{\sum_{i=1}^{1000000}{\exp{(s_i)}}}
$$

<img src="./images/cbow_big2.png" width="50%" height="50%" />

### 4.2.2 다중 분류에서 이진 분류로

Negative Sampling(NEG)의 핵심 아이디어는 **'이진 분류'**<sup>binary classificaton</sup>에 있다. 즉, '다중 분류<sup>multi-class classification</sup>'를 '이진 분류'로 근사하는 것이 Negative Sampling을 이해하는 데 중요한 포인트다.

![](./images/neg.png)

### 4.2.3 시그모이드 함수와 교차 엔트로피 오차

다중 분류의 경우에는 출력층에서 점수<sup>score</sup>를 확률로 변환할 때, 소프트맥스 함수를 사용하고, 이진 분류의 경우에는 시그모이드 함수를 사용한다. 


$$
y = \frac{1}{1 + \exp{(-x)}}
$$

시그모이드 함수를 적용해 확률 $y$를 구한 후, 이 확률 $y$로 부터 손실(Loss)을 구한다.


$$
L = - \left[ t \log{y} + (1-t) \log{(1-y)} \right]
$$

'Sigmoid with Loss' 레이어의 역전파를 구하면 다음과 같다.

$$
\begin{align*}
\frac{\partial L}{\partial x} &= \frac{\partial L}{\partial y} \frac{\partial y}{\partial x} = y-t \\
\frac{\partial L}{\partial y} &= -\frac{t}{y} + \frac{1-t}{1-y} = \frac{y-t}{y(1-y)} \\
\frac{\partial y}{\partial x} &= y(1-y)
\end{align*}
$$

<img src="./images/sigmoid02.png" width="70%" height="70%" />

위의 식에서 'Sigmoid with Loss'레이어에서의 역전파는 $y-t$ 즉, 오차가 앞의 계층으로 흘러가게 된다. 따라서, 오차가 크면 '크게'학습하고, 오차가 작으면 '작게'학습하게 된다.

### 4.2.4 다중 분류에서 이진 분류로(구현)

![](./images/cbow.png)

In [1]:
# Chap04/negative_sampling_layer.py
import sys
sys.path.append('..')
import collections
from common.np import *
from common.layers import Embedding, SigmoidWithLoss

In [9]:
class EmbeddingDot:
    def __init__(self, W):
        self.embed = Embedding(W)
        self.params = self.embed.params
        self.grads = self.embed.grads
        self.cache = None
        
    def forward(self, h, idx):
        target_W = self.embed.forward(idx)
        out = np.sum(target_W * h, axis=1)
        
        self.cache = (h, target_W)
        return out
    
    def backward(self, dout):
        h, target_W = self.cache
        dout = dout.reshape(dout.shape[0], 1)
        
        dtarget_W = dout * h
        self.embed.backward(dtarget_W)
        dh = dout * target_W
        return dh